In [52]:
import os

In [53]:
%pwd

'd:\\Repositories\\ML-geo'

In [54]:
os.chdir("d:/Repositories/ML-geo/")

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    transformation_path: Path
    model_name_1: str
    model_name_2: str
    reg_alpha: float
    reg_lambda: float
    target_column_1: float
    target_column_2: float

In [56]:
from XGboost_for_slopes.constants import *
from XGboost_for_slopes.utils.common import read_yaml, create_directories

In [57]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Xgboost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path  = config.test_data_path,
            transformation_path = config.transformation_path,
            model_name_1 = config.model_name_1,
            model_name_2 = config.model_name_2,
            reg_alpha = params.REG_ALPHA,
            reg_lambda= params.REG_LAMBDA,
            target_column_1= schema.target_1,
            target_column_2= schema.target_2
            )
        return model_trainer_config
    



In [63]:
import os 
from XGboost_for_slopes import logger
import pandas as pd
import xgboost as xgb
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from XGboost_for_slopes.utils.common import save_json

In [64]:
class ModelTrainer():
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def get_model_trainer_object(self):

        """
        This function is responsible for training the model
        """
        train_data_df      = pd.read_csv(self.config.train_data_path)
        test_data_df      = pd.read_csv(self.config.test_data_path)

        Transform_pipeline = joblib.load(self.config.transformation_path)

        
        y_data_train_1 = train_data_df[self.config.target_column_1]
        X_data_train_1 = train_data_df.drop([self.config.target_column_1, self.config.target_column_2], axis = 1)


        X_data_train_1 = Transform_pipeline.fit_transform(X_data_train_1)


        params_xgb  =  {"reg_alpha": self.config.reg_alpha,
                        "reg_lambda": self.config.reg_lambda}
    
        xgb_Reg = xgb.XGBRegressor(**params_xgb, booster= "gblinear")
        xgb_Reg.fit(X_data_train_1,y_data_train_1)


        joblib.dump(xgb_Reg, os.path.join(self.config.root_dir,self.config.model_name_1))

        logger.info(f"Model_1 is saved in {os.path.join(self.config.root_dir,self.config.model_name_1)}")


        y_data_train_2 = train_data_df[self.config.target_column_2]
        X_data_train_2 = train_data_df.drop([self.config.target_column_1, self.config.target_column_2], axis = 1)


        X_data_train_2 = Transform_pipeline.fit_transform(X_data_train_2)
    
        xgb_Reg = xgb.XGBRegressor(**params_xgb, booster= "gblinear")
        xgb_Reg.fit(X_data_train_2,y_data_train_2)


        joblib.dump(xgb_Reg, os.path.join(self.config.root_dir,self.config.model_name_2))

        logger.info(f"Model_2 is saved in {os.path.join(self.config.root_dir,self.config.model_name_2)}")


        







In [62]:
train_data_df      = pd.read_csv("artifacts/data_transformation/train.csv")
test_data_df       = pd.read_csv("artifacts/data_transformation/test.csv")

params_xgb  =  {"reg_alpha": 0.01,
                "reg_lambda": 0.03}

Transform_pipeline = joblib.load("artifacts/data_transformation/transformation.pkl")


y_data_train_1 = train_data_df["FSs"]
X_data_train_1 = train_data_df.drop(["FSs","FSgmp"], axis = 1)

y_data_test_1 = test_data_df["FSs"]
X_data_test_1 = test_data_df.drop(["FSs","FSgmp"], axis = 1)

X_data_train_1 = Transform_pipeline.fit_transform(X_data_train_1)
X_data_test_1  = Transform_pipeline.fit_transform(X_data_test_1)

xgb_Reg = xgb.XGBRegressor(**params_xgb, booster = "gblinear")


xgb_Reg.fit(X_data_train_1,y_data_train_1)
r2_value = r2_score(y_data_test_1, xgb_Reg.predict(X_data_test_1))
print(r2_value)

0.9431403953365518


In [65]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.get_model_trainer_object()

except Exception as e:
    raise e

[2023-12-07 12:51:40,897: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-07 12:51:40,898: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-07 12:51:40,902: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-07 12:51:40,903: INFO: common: created directory at: artifacts]
[2023-12-07 12:51:40,905: INFO: common: created directory at: artifacts/model_trainer]
[2023-12-07 12:51:41,346: INFO: 4112445815: Model_1 is fitted, r2 score for testing data is: 0.9446852144372012]
[2023-12-07 12:51:41,349: INFO: 4112445815: Model_1 is saved in artifacts/model_trainer\model_1.joblib]
[2023-12-07 12:51:41,756: INFO: 4112445815: Model_2 is fitted, r2 score for testing data is: 0.9448023938791724]
[2023-12-07 12:51:41,759: INFO: 4112445815: Model_2 is saved in artifacts/model_trainer\model_2.joblib]
[2023-12-07 12:51:41,760: INFO: common: json file saved at: scores.json]
